In [2]:
# !pip install OpenPermID

In [70]:
from OpenPermID import OpenPermID
import pandas as pd
import time
from random import randint
from time import sleep

In [71]:
# pass in key
%run ./perm_id_key.ipynb

In [75]:
# initiate OpenPermID
opid = OpenPermID()
opid.set_access_token(access_token)

In [72]:
# import the contents of CSPP_PEPP_corporate_bond_holdings_20210430 into a dataframe
corporate_bond_holdings_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_PEPP_corporate_bond_holdings_20210430.xlsx')

In [73]:
# remove accents from companies in ISSUER column
corporate_bond_holdings_df['ISSUER'] = corporate_bond_holdings_df['ISSUER'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [123]:
# create a list of unique ISSUERS to run through the API
unique_issuers = corporate_bond_holdings_df["ISSUER"].unique()
unique_issuers

array(['Elia Transmission Belgium', 'Enodia SCRL',
       'Groupe Bruxelles Lambert SA', 'Fluvius System Operator CVBA',
       'BEFIMMO SA', 'Elia Group SA', 'Anheuser-Busch InBev N.V./S.A.',
       'Brussels Airport Company SA/NV', 'Apetra N.V.',
       'Barry Callebaut Services N.V', 'Solvay S.A.',
       'Lonza Finance International NV', 'ENI Finance International S.A.',
       'VGP NV', 'JAB Holdings B.V.', 'TRATON Finance Luxemburg SA',
       'SIX Finance (Luxembourg) S.A.', 'Stellantis N.V, Amsterdam',
       'Brisa - Concesso Rodoviria, SA', 'EDP - Energias de Portugal SA',
       'Metropolitano de Lisboa', 'NOS SGPS, Lisbon',
       'Telecom Italia Finance S.A.', 'Nederlandse Gasunie, N.V.',
       'EDP Finance B.V.', 'TenneT Holding BV', 'ENEXIS Holding NV',
       'Alliander N.V.', 'Akzo Nobel N.V.', 'Nestl Finance Intl Ltd.',
       'Wolters Kluwer N.V.', 'ASML Holding N.V.',
       'Zpadoslovensk energetika AS', 'Holcim Finance (Luxembg) S.A.',
       'Heineken N.V.', 'Ko

In [298]:
len(unique_issuers)

360

In [140]:
# create a test run to make sure the function is working properly
test_run = unique_issuers[:4]
test_run

In [162]:
# create a list to add the data frames from the name search
companies_list = []

In [163]:
# function for using the unique_issuers to find an ID number by name search
def search_company_name(unique_companies):

    for company in unique_companies:
        # this is setup to retry if the search doesn't work on the first time
        # upon using the website I noticed that it took a few times upon manual entry
        # for the search results to appear
        for attempt in range(3):
            try:
                # first attempt
                output,err = opid.search(company)
                companies_list.append(output["organizations"])
                print("df added try 1")
            except:
                # next attempt
                print("extra break hold on a minute")
                # add an extra break
                time.sleep(10)
                output,err = opid.search(company)
                companies_list.append(output["organizations"])
                print("df added on try 2")
            else:
                break
        else:
            print(company)
            print("there were no entries for this company")
            pass
        time.sleep(10)
    print("done")
    

In [164]:
# run the search by name function
search_company_name(unique_issuers)

df added try 1
df added try 1
extra break hold on a minute
df added on try 2
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
extra break hold on a minute
df added on try 2
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df added try 1
df add

In [166]:
# take the list of individual dataframes and concatenate them into one
companies_df = pd.concat(companies_list)

In [169]:
# save the df to csv file
# companies_df.to_csv (r'/Users/gabbyvinco/Desktop/companies.csv', index = False, header=True)

In [303]:
# companies_df.head(50)

In [182]:
# sort by index because for each search term there were up to 5 responses. by sorting all of the first responses
# will have an index of 0. However there were only about 331 values with index of 0, so I decided it would be best
# to keep all the indexes up until 2 (the first 3 responses from the search term, which is up until the 615 row)
companies_df = companies_df.sort_index()

In [304]:
companies_df = companies_df.reset_index()
# companies_df

In [202]:
# keep the first 615 rows
companies_df = companies_df.iloc[:614,:]

In [222]:
# create a tester list for the extra company info function
tester = companies_df.iloc[:4,:]

In [243]:
# create a new list to add the outputs fromt he extra company info function
companies_info = []

In [244]:
# function for using the companies_df to take the perm id number and get more info about the company
def grab_extra_company_info(df):
    for item in range(len(df)):
        for attempt in range(3):
            # another multiple attempts feature included in this lookup
            try:
                # first attempt
                perm_id = companies_df['@id'].iloc[item][-12:]
                output,err = opid.lookup(perm_id, format="dataframe")
                companies_info.append(output)
                print("extra info added try 1")
            except:
                # second attempt
                print("extra break hold on a minute")
                time.sleep(10)
                perm_id = companies_df['@id'].iloc[item][-12:]
                output,err = opid.lookup(perm_id, format="dataframe")
                companies_info.append(output)
                print("extra info added on try 2")
            else:
                break
        else:
            print(company)
            print("there was no extra info for this company")
            pass
        time.sleep(10)
    print("done")

In [245]:
# run the extra info function 
grab_extra_company_info(companies_df)

extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info 

extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra break hold on a minute
extra info added on try 2
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
extra info added try 1
ex

In [247]:
# concatenate all of the dataframes from companies_info list into one dataframe
companies_info_df = pd.concat(companies_info)

In [306]:
# companies_info_df

In [305]:
companies_info_df = companies_info_df.reset_index(drop=True)
companies_info_df

,@id,@type,tr-common:hasPermId,mdaas:HeadquartersAddress,mdaas:RegisteredAddress,hasActivityStatus,tr-org:hasLEI,hasLatestOrganizationFoundedDate,hasPrimaryBusinessSector,hasPrimaryEconomicSector,...,tr-org:hasHeadquartersPhoneNumber,tr-org:hasRegisteredPhoneNumber,hasURL,hasOrganizationPrimaryQuote,hasPrimaryInstrument,tr-org:hasHeadquartersFaxNumber,hasHoldingClassification,hasIPODate,tr-org:hasRegisteredFaxNumber,hasInactiveDate
0,https://permid.org/1-5071549731,tr-org:Organization,5071549731,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,tr-org:statusActive,549300A3EZXECDLW2V25,2019-07-31T00:00:00Z,https://permid.org/1-4294952820,https://permid.org/1-4294952821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://permid.org/1-5057817730,tr-org:Organization,5057817730,c/o Deutsche Bahn Aktiengesellschaft\nEuropapl...,c/o Deutsche Bahn Aktiengesellschaft\nEuropapl...,tr-org:statusActive,52990002BAIDUAIIYU29,2017-09-01T00:00:00Z,https://permid.org/1-4294952739,https://permid.org/1-4294952740,...,493029764006,493029764006,https://ir.deutschebahn.com/de/anleihen-rating...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://permid.org/1-4296543170,tr-org:Organization,4296543170,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,tr-org:statusActive,529900ERDFHS6C1M4U58,NaN,https://permid.org/1-4294952825,https://permid.org/1-4294952723,...,NaN,NaN,https://www.telekom.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://permid.org/1-4296653814,tr-org:Organization,4296653814,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,tr-org:statusActive,5299005PIU82TA2PW123,2001-02-14T00:00:00Z,https://permid.org/1-4294952820,https://permid.org/1-4294952821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://permid.org/1-4296930961,tr-org:Organization,4296930961,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,tr-org:statusActive,724500CNCIO1ZTJ0X675,2001-04-03T00:00:00Z,https://permid.org/1-4294952820,https://permid.org/1-4294952821,...,NaN,NaN,https://www.enbw.com/company/investors/bonds/e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,https://permid.org/1-4296680912,tr-org:Organization,4296680912,"Rua Lauro Mullerm, 116 Sala 3902 e 3903\nRio D...",NaN,tr-org:statusActive,NaN,1995-01-01T00:00:00Z,https://permid.org/1-4294952773,https://permid.org/1-4294952774,...,552125080505,NaN,http://www.lorentzen.com.br,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,https://permid.org/1-4296858525,tr-org:Organization,4296858525,"Challenger\n1, avenue Eugene Freyssinet\nSAINT...",Challenger\n1 Av Eugene Freyssinet\nGUYANCOURT...,tr-org:statusActive,969500IL7U7UEI5ZYJ34,1982-07-01T00:00:00Z,https://permid.org/1-4294952762,https://permid.org/1-4294952767,...,33130603300,33130603300,http://www.bouygues-construction.com,NaN,NaN,33130604861,NaN,NaN,33130604861,NaN
450,https://permid.org/1-5067178704,tr-org:Organization,5067178704,Hungary\n,NaN,tr-org:statusActive,NaN,NaN,https://permid.org/1-4294952860,https://permid.org/1-1004365438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,https://permid.org/1-4296730050,tr-org:Organization,4296730050,25 Rue De Clichy\nPARIS\nILE-DE-FRANCE\n75009\...,25 Rue De Clichy\nParis 9\nPARIS\nILE-DE-FRANC...,tr-org:statusActive,549300850GMPCX5CK282,2006-02-28T00:00:00Z,https://permid.org/1-4294952739,https://permid.org/1-4294952740,...,33140758000,NaN,NaN,NaN,NaN,33145635728,NaN,NaN,NaN,NaN


In [250]:
# save the df to csv file
# companies_info_df.to_csv (r'/Users/gabbyvinco/Desktop/companies_info.csv', index = False, header=True)

In [274]:
# only keep companies whose buisness sector row is not NaN
companies_info_df = companies_info_df[companies_info_df['hasPrimaryBusinessSector'].notna()]

In [302]:
# create a test_list for the buisness sector function
test_list = companies_info_df.iloc[:4,:]
# test_list

In [276]:
# create a new list to hold the information about what buiness sector the companies are in
industry_info = []

In [277]:
# function to find out industry info
def grab_sector_info(df):
    for item in range(len(df)):
        for attempt in range(3):
            # another multiple attempts feature included in this lookup
            try:
                # first attempt
                business_sector_id = companies_info_df['hasPrimaryBusinessSector'].iloc[item][-12:]
                output,err = opid.lookup(business_sector_id, format="dataframe")
                industry_info.append(output)
                print("sector info added try 1")
            except:
                # second attempt
                print("extra break hold on a minute")
                time.sleep(10)
                business_sector_id = companies_info_df['hasPrimaryBusinessSector'].iloc[item][-12:]
                output,err = opid.lookup(business_sector_id, format="dataframe")
                industry_info.append(output)
                print("sector info added on try 2")
            else:
                break
        else:
            print(company)
            print("there was no sector info for this company")
            pass
        time.sleep(10)
    print("done")

In [278]:
# run the function to get the industry info
grab_sector_info(companies_info_df)

sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info adde

sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info added try 1
sector info adde

In [279]:
# concatenate the industry info dataframes into one
company_sectors_df = pd.concat(industry_info)
# company_sectors_df

In [366]:
company_sectors_df = company_sectors_df.reset_index(drop=True)
# company_sectors_df

In [281]:
# save the df to csv file
# company_sectors_df.to_csv (r'/Users/gabbyvinco/Desktop/company_sectors.csv', index = False, header=True)

In [313]:
# rename the @id column in company_sectors_df to match the column name in companies_info_df ('hasPrimaryBusinessSector')
company_sectors_df = company_sectors_df.rename(columns={'@id': 'hasPrimaryBusinessSector'})
# company_sectors_df

In [310]:
# merge company_sectors_df into companies_info_df
full_company_info_df = pd.concat([company_sectors_df, companies_info_df.reindex(companies_info_df.index)], axis=1)
# full_company_info_df.head()

In [326]:
complete_companies_df = full_company_info_df[['@id','@type','tr-common:hasPermId','trbc:BusinessClassificationCode','rdfs:label','prefLabel','rdfs:comment','hasPrimaryBusinessSector','hasPrimaryEconomicSector','tr-org:hasLEI','mdaas:HeadquartersAddress','tr-org:hasHeadquartersPhoneNumber','hasURL']].copy()
# complete_companies_df.head()

In [336]:
complete_companies_df['type_keep'] = complete_companies_df.iloc[:,2]
complete_companies_df['org_perm_id'] = complete_companies_df.iloc[:,4]
complete_companies_df['sector_perm_id'] = complete_companies_df.iloc[:,3]

In [338]:
complete_companies_df = complete_companies_df.drop(['@type', 'tr-common:hasPermId','hasPrimaryBusinessSector'], axis=1)

In [341]:
complete_companies_df.head()

,@id,trbc:BusinessClassificationCode,rdfs:label,prefLabel,rdfs:comment,hasPrimaryEconomicSector,tr-org:hasLEI,mdaas:HeadquartersAddress,tr-org:hasHeadquartersPhoneNumber,hasURL,type_keep,org_perm_id,sector_perm_id
0,https://permid.org/1-5071549731,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,549300A3EZXECDLW2V25,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,NaN,NaN,tr-org:Organization,5071549731,4294952820
2,https://permid.org/1-4296543170,5740,Telecommunications Services,Telecommunications Services,Owners and operators of fixed line and wireles...,https://permid.org/1-4294952723,529900ERDFHS6C1M4U58,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,NaN,https://www.telekom.com/,tr-org:Organization,4296543170,4294952825
3,https://permid.org/1-4296653814,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,5299005PIU82TA2PW123,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,NaN,NaN,tr-org:Organization,4296653814,4294952820
4,https://permid.org/1-4296930961,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,724500CNCIO1ZTJ0X675,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,NaN,https://www.enbw.com/company/investors/bonds/e...,tr-org:Organization,4296930961,4294952820
5,https://permid.org/1-4296072717,5510,Banking & Investment Services,Banking & Investment Services,"Providers of retail, commercial and investment...",https://permid.org/1-4294952740,549300TG53AWJ719M814,Kabelweg 37\nAMSTERDAM\nNOORD-HOLLAND\n1014 BA...,NaN,https://www.eon.com/de/investoren/anleihen/e-d...,tr-org:Organization,4296072717,4294952739


In [347]:
companies_firstresult_df = companies_df[companies_df['index'] == 0]

In [350]:
companies_firstresult_df = companies_firstresult_df.drop(['index'], axis=1)
companies_firstresult_df

,@id,organizationName,orgSubtype,hasURL,primaryTicker,hasHoldingClassification
0,https://permid.org/1-5071549731,Elia Transmission Belgium NV,Company,NaN,NaN,NaN
1,https://permid.org/1-5057817730,Deutsche Bahn Finance GmbH,Special Purpose Vehicle,https://ir.deutschebahn.com/de/anleihen-rating...,NaN,NaN
2,https://permid.org/1-4296543170,Deutsche Telekom International Finance BV,Special Purpose Vehicle,https://www.telekom.com/,NaN,NaN
3,https://permid.org/1-4296653814,innogy Finance BV,Special Purpose Vehicle,NaN,NaN,NaN
4,https://permid.org/1-4296930961,EnBW International Finance BV,Special Purpose Vehicle,https://www.enbw.com/company/investors/bonds/e...,NaN,NaN
...,...,...,...,...,...,...
327,https://permid.org/1-5042958012,Euronext NV,Company,https://www.euronext.com/,ENX,publiclyHeld
328,https://permid.org/1-4295867357,Dassault Systemes SE,Company,https://www.3ds.com,DSY,publiclyHeld
329,https://permid.org/1-4298361219,DNA Oyj,Company,https://www.dna.fi/,DNA,NaN
330,https://permid.org/1-4295867538,Vinci SA,Company,https://www.vinci.com/,DG,publiclyHeld


In [367]:
# join complete_companies_df and companies_firstresult_df
# final_df = pd.concat([complete_companies_df, companies_firstresult_df], axis=1, join="inner")

# or use companies_df instead of companies_firstresult_df
final_df = pd.concat([complete_companies_df, companies_df], axis=1, join="inner")

In [369]:
final_df

,@id,trbc:BusinessClassificationCode,rdfs:label,prefLabel,rdfs:comment,hasPrimaryEconomicSector,tr-org:hasLEI,mdaas:HeadquartersAddress,tr-org:hasHeadquartersPhoneNumber,hasURL,...,org_perm_id,sector_perm_id,level_0,index,@id,organizationName,orgSubtype,hasURL,primaryTicker,hasHoldingClassification
0,https://permid.org/1-5071549731,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,549300A3EZXECDLW2V25,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,NaN,NaN,...,5071549731,4294952820,0,0,https://permid.org/1-5071549731,Elia Transmission Belgium NV,Company,NaN,NaN,NaN
2,https://permid.org/1-4296543170,5740,Telecommunications Services,Telecommunications Services,Owners and operators of fixed line and wireles...,https://permid.org/1-4294952723,529900ERDFHS6C1M4U58,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,NaN,https://www.telekom.com/,...,4296543170,4294952825,2,0,https://permid.org/1-4296543170,Deutsche Telekom International Finance BV,Special Purpose Vehicle,https://www.telekom.com/,NaN,NaN
3,https://permid.org/1-4296653814,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,5299005PIU82TA2PW123,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,NaN,NaN,...,4296653814,4294952820,3,0,https://permid.org/1-4296653814,innogy Finance BV,Special Purpose Vehicle,NaN,NaN,NaN
4,https://permid.org/1-4296930961,5910,Utilities,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,724500CNCIO1ZTJ0X675,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,NaN,https://www.enbw.com/company/investors/bonds/e...,...,4296930961,4294952820,4,0,https://permid.org/1-4296930961,EnBW International Finance BV,Special Purpose Vehicle,https://www.enbw.com/company/investors/bonds/e...,NaN,NaN
5,https://permid.org/1-4296072717,5510,Banking & Investment Services,Banking & Investment Services,"Providers of retail, commercial and investment...",https://permid.org/1-4294952740,549300TG53AWJ719M814,Kabelweg 37\nAMSTERDAM\nNOORD-HOLLAND\n1014 BA...,NaN,https://www.eon.com/de/investoren/anleihen/e-d...,...,4296072717,4294952739,5,0,https://permid.org/1-4296072717,E On International Finance BV,Special Purpose Vehicle,https://www.eon.com/de/investoren/anleihen/e-d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,https://permid.org/1-4296680912,5120,Mineral Resources,Mineral Resources,"Miners and processors of steel, aluminum, prec...",https://permid.org/1-4294952774,NaN,"Rua Lauro Mullerm, 116 Sala 3902 e 3903\nRio D...",552125080505,http://www.lorentzen.com.br,...,4296680912,4294952773,448,1,https://permid.org/1-4296716646,Reseaux Cables De France,Company,NaN,NaN,NaN
449,https://permid.org/1-4296858525,5220,Industrial & Commercial Services,Industrial & Commercial Services,"Providers of construction, engineering, commer...",https://permid.org/1-4294952767,969500IL7U7UEI5ZYJ34,"Challenger\n1, avenue Eugene Freyssinet\nSAINT...",33130603300,http://www.bouygues-construction.com,...,4296858525,4294952762,449,1,https://permid.org/1-4296716646,Reseaux Cables De France,Company,NaN,NaN,NaN
450,https://permid.org/1-5067178704,6010,Real Estate,Real Estate,Developers and managers of real estate operati...,https://permid.org/1-1004365438,NaN,Hungary\n,NaN,NaN,...,5067178704,4294952860,450,1,https://permid.org/1-4297270573,Befimmo Property Services SA,Company,NaN,NaN,NaN
451,https://permid.org/1-4296730050,5510,Banking & Investment Services,Banking & Investment Services,"Providers of retail, commercial and investment...",https://permid.org/1-4294952740,549300850GMPCX5CK282,25 Rue De Clichy\nPARIS\nILE-DE-FRANCE\n75009\...,33140758000,NaN,...,4296730050,4294952739,451,1,https://permid.org/1-5074666524,Brussels Airport Terminal Co SA NV,Unknown,NaN,NaN,NaN


In [370]:
final_df['org_url'] = final_df.iloc[:,9]
final_df['id'] = final_df.iloc[:,0]

In [371]:
final_df = final_df.drop(['@id', 'rdfs:label','hasURL','type_keep'], axis=1)

In [372]:
final_df.head()

,trbc:BusinessClassificationCode,prefLabel,rdfs:comment,hasPrimaryEconomicSector,tr-org:hasLEI,mdaas:HeadquartersAddress,tr-org:hasHeadquartersPhoneNumber,org_perm_id,sector_perm_id,level_0,index,organizationName,orgSubtype,primaryTicker,hasHoldingClassification,org_url,id
0,5910,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,549300A3EZXECDLW2V25,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,NaN,5071549731,4294952820,0,0,Elia Transmission Belgium NV,Company,NaN,NaN,NaN,https://permid.org/1-5071549731
2,5740,Telecommunications Services,Owners and operators of fixed line and wireles...,https://permid.org/1-4294952723,529900ERDFHS6C1M4U58,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,NaN,4296543170,4294952825,2,0,Deutsche Telekom International Finance BV,Special Purpose Vehicle,NaN,NaN,https://www.telekom.com/,https://permid.org/1-4296543170
3,5910,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,5299005PIU82TA2PW123,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,NaN,4296653814,4294952820,3,0,innogy Finance BV,Special Purpose Vehicle,NaN,NaN,NaN,https://permid.org/1-4296653814
4,5910,Utilities,"Producers and distributors of electricity, nat...",https://permid.org/1-4294952821,724500CNCIO1ZTJ0X675,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,NaN,4296930961,4294952820,4,0,EnBW International Finance BV,Special Purpose Vehicle,NaN,NaN,https://www.enbw.com/company/investors/bonds/e...,https://permid.org/1-4296930961
5,5510,Banking & Investment Services,"Providers of retail, commercial and investment...",https://permid.org/1-4294952740,549300TG53AWJ719M814,Kabelweg 37\nAMSTERDAM\nNOORD-HOLLAND\n1014 BA...,NaN,4296072717,4294952739,5,0,E On International Finance BV,Special Purpose Vehicle,NaN,NaN,https://www.eon.com/de/investoren/anleihen/e-d...,https://permid.org/1-4296072717


In [373]:
cols = list(final_df.columns)
reordered = ['organizationName',
             'org_perm_id',
             'id',
             'orgSubtype',
             'prefLabel',
             'rdfs:comment',
             'sector_perm_id',
             'trbc:BusinessClassificationCode',
             'hasPrimaryEconomicSector',
             'mdaas:HeadquartersAddress',
             'tr-org:hasHeadquartersPhoneNumber',
             'org_url',
             'primaryTicker',
             'hasHoldingClassification',
             'tr-org:hasLEI']

In [374]:
final_df = final_df[reordered]

In [377]:
final_df = final_df.rename(columns={'org_perm_id': 'orgID', 'id': 'orgID_URL','prefLabel': 'businessSector','rdfs:comment':'busSect_description', 'sector_perm_id':'busSectID', 'trbc:BusinessClassificationCode':'Business_ClassCode', 'hasPrimaryEconomicSector':'EconomicSectID_URL','mdaas:HeadquartersAddress':'HQ_Address', 'tr-org:hasHeadquartersPhoneNumber':'HQ_PhoneNumber','org_url':'orgURL','tr-org:hasLEI':'LEI'})

In [378]:
final_df

,organizationName,orgID,orgID_URL,orgSubtype,businessSector,busSect_description,busSectID,Business_ClassCode,EconomicSectID_URL,HQ_Address,HQ_PhoneNumber,orgURL,primaryTicker,hasHoldingClassification,LEI
0,Elia Transmission Belgium NV,5071549731,https://permid.org/1-5071549731,Company,Utilities,"Producers and distributors of electricity, nat...",4294952820,5910,https://permid.org/1-4294952821,Boulevard De L'empereur 20\nBRUXELLES\nBRUXELL...,NaN,NaN,NaN,NaN,549300A3EZXECDLW2V25
2,Deutsche Telekom International Finance BV,4296543170,https://permid.org/1-4296543170,Special Purpose Vehicle,Telecommunications Services,Owners and operators of fixed line and wireles...,4294952825,5740,https://permid.org/1-4294952723,Stationsplein 8 K\nMAASTRICHT\nLIMBURG\n6221BT...,NaN,https://www.telekom.com/,NaN,NaN,529900ERDFHS6C1M4U58
3,innogy Finance BV,4296653814,https://permid.org/1-4296653814,Special Purpose Vehicle,Utilities,"Producers and distributors of electricity, nat...",4294952820,5910,https://permid.org/1-4294952821,Willemsplein 4\nS-HERTOGENBOSCH\nNOORD-BRABANT...,NaN,NaN,NaN,NaN,5299005PIU82TA2PW123
4,EnBW International Finance BV,4296930961,https://permid.org/1-4296930961,Special Purpose Vehicle,Utilities,"Producers and distributors of electricity, nat...",4294952820,5910,https://permid.org/1-4294952821,Herikerbergweg 122\nAMSTERDAM\nNOORD-HOLLAND\n...,NaN,https://www.enbw.com/company/investors/bonds/e...,NaN,NaN,724500CNCIO1ZTJ0X675
5,E On International Finance BV,4296072717,https://permid.org/1-4296072717,Special Purpose Vehicle,Banking & Investment Services,"Providers of retail, commercial and investment...",4294952739,5510,https://permid.org/1-4294952740,Kabelweg 37\nAMSTERDAM\nNOORD-HOLLAND\n1014 BA...,NaN,https://www.eon.com/de/investoren/anleihen/e-d...,NaN,NaN,549300TG53AWJ719M814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,Reseaux Cables De France,4296680912,https://permid.org/1-4296680912,Company,Mineral Resources,"Miners and processors of steel, aluminum, prec...",4294952773,5120,https://permid.org/1-4294952774,"Rua Lauro Mullerm, 116 Sala 3902 e 3903\nRio D...",552125080505,http://www.lorentzen.com.br,NaN,NaN,NaN
449,Reseaux Cables De France,4296858525,https://permid.org/1-4296858525,Company,Industrial & Commercial Services,"Providers of construction, engineering, commer...",4294952762,5220,https://permid.org/1-4294952767,"Challenger\n1, avenue Eugene Freyssinet\nSAINT...",33130603300,http://www.bouygues-construction.com,NaN,NaN,969500IL7U7UEI5ZYJ34
450,Befimmo Property Services SA,5067178704,https://permid.org/1-5067178704,Company,Real Estate,Developers and managers of real estate operati...,4294952860,6010,https://permid.org/1-1004365438,Hungary\n,NaN,NaN,NaN,NaN,NaN
451,Brussels Airport Terminal Co SA NV,4296730050,https://permid.org/1-4296730050,Unknown,Banking & Investment Services,"Providers of retail, commercial and investment...",4294952739,5510,https://permid.org/1-4294952740,25 Rue De Clichy\nPARIS\nILE-DE-FRANCE\n75009\...,33140758000,NaN,NaN,NaN,549300850GMPCX5CK282


In [379]:
# save the df to csv file
# final_df.to_csv (r'/Users/gabbyvinco/Desktop/companies_final_451.csv', index = False, header=True)